In [2]:
# Linear Regression project with generalized input dimensions and L2 normalisation
__author__ = "Billy Cao"
import tensorflow as tf
import generator

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)  # change to false when running on ML server
tf.keras.mixed_precision.set_global_policy('mixed_float16')

batchSize = 100
gen = generator.gen2d(batchSize)  # gen2d for 2D input
epochs = 1000
regTerm = 0.001
learning_rate = 1e-5


class LinearModel:  # initializing to 1 now, but also can do 0 or random
    def __call__(self, x):  # predicting function
        return self.Weight * x + self.Bias

    def __init__(self):
        self.Weight = tf.Variable(1.0, shape=tf.TensorShape(None))  # initialize m to any shape
        self.Bias = tf.Variable(1.0)


def loss(y, pred):  # Mean Squared Error with L2 Normalisation
    return tf.reduce_mean(tf.square(y - pred)) + tf.reduce_sum(regTerm * tf.square(linear_model.Weight))


def train(linear_model, x, y, lr):
    # use to reshape into 2D vector if input is of unknown dimension
    # if len(x.shape) == 1:
    #     X = tf.reshape(x, [x.shape[0], 1])
    with tf.GradientTape(persistent=False) as t:  # persistent=True is needed if assigning dy_dWeight, dy_dBias in 2 lines. Limits the times u can call it to once
        current_loss = loss(y, linear_model(x))
    dy_dWeight, dy_dBias = t.gradient(current_loss, [linear_model.Weight, linear_model.Bias])
    linear_model.Weight.assign_sub(lr * dy_dWeight)
    linear_model.Bias.assign_sub(lr * dy_dBias)


linear_model = LinearModel()
sampleX, sampleY = next(gen)
linear_model.Weight.assign([1.0] * sampleX.shape[-1])  # initialize m to 1.0 and make it same dimension as the input
for epoch_count in range(epochs):
    x, y = next(gen)
    real_loss = loss(y, linear_model(x))
    train(linear_model, x, y, lr=learning_rate)
    print(f"Epoch count {epoch_count}: Loss: {real_loss.numpy()}")

print(f'Weight: {linear_model.Weight.numpy()}, Bias: {linear_model.Bias.numpy()}')


Epoch count 0: Loss: 2015.9395751953125
Epoch count 1: Loss: 1947.5098876953125
Epoch count 2: Loss: 2013.4190673828125
Epoch count 3: Loss: 1865.3358154296875
Epoch count 4: Loss: 1900.5662841796875
Epoch count 5: Loss: 1767.779052734375
Epoch count 6: Loss: 1478.0068359375
Epoch count 7: Loss: 1624.0269775390625
Epoch count 8: Loss: 1540.7105712890625
Epoch count 9: Loss: 1542.34033203125
Epoch count 10: Loss: 1224.596435546875
Epoch count 11: Loss: 1380.0296630859375
Epoch count 12: Loss: 1236.3336181640625
Epoch count 13: Loss: 1215.1187744140625
Epoch count 14: Loss: 1253.491943359375
Epoch count 15: Loss: 994.7985229492188
Epoch count 16: Loss: 919.5164184570312
Epoch count 17: Loss: 913.7876586914062
Epoch count 18: Loss: 948.9878540039062
Epoch count 19: Loss: 893.8351440429688
Epoch count 20: Loss: 874.4326171875
Epoch count 21: Loss: 934.5164794921875
Epoch count 22: Loss: 904.7742309570312
Epoch count 23: Loss: 902.1901245117188
Epoch count 24: Loss: 768.997314453125
Epoch c